In [1]:
# کلون کردن ریپوزیتوری
!git clone https://github.com/Peiyance/TransGNN-torch.git

# انتقال دایرکتوری Data به مسیر اصلی
!mv /content/TransGNN-torch/Data /content/Data

# بررسی وجود فایل‌ها
!ls /content/Data
!ls /content/Data/yelp
!ls /content/Data/gowalla
!ls /content/Data/tmall

Cloning into 'TransGNN-torch'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 101 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 22.43 MiB | 4.55 MiB/s, done.
Resolving deltas: 100% (18/18), done.
gowalla  Readme.md  tmall  yelp
item_list.txt  test.txt   trnMat.pkl  user_list.txt
process.py     train.txt  tstMat.pkl
item_list.txt  test.txt   trnMat.pkl  user_list.txt
process.py     train.txt  tstMat.pkl
tstMat.pkl


In [ ]:
gowalla  tmall  yelp
item_list.txt  process.py  test.txt  train.txt  trnMat.pkl  tstMat.pkl  user_list.txt  # برای yelp و gowalla
tstMat.pkl  # برای tmall

In [3]:
# نصب پکیج‌های مورد نیاز
!pip install torch torch-geometric setproctitle

import os
import datetime
import argparse
import pickle
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from scipy.sparse import csr_matrix, coo_matrix, dok_matrix
import torch_geometric.nn as pygnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# تعریف آرگومان‌ها
def ParseArgs():
    parser = argparse.ArgumentParser(description='Model Params')
    parser.add_argument('--lr', default=1e-3, type=float, help='learning rate')
    parser.add_argument('--batch', default=4096, type=int, help='batch size')
    parser.add_argument('--leaky', default=0.5, type=float, help='slope of leaky relu')
    parser.add_argument('--tstBat', default=64, type=int, help='number of users in a testing batch')
    parser.add_argument('--epoch', default=40, type=int, help='number of epochs')
    parser.add_argument('--decay', default=1, type=float, help='weight decay rate')
    parser.add_argument('--save_path', default='tem', help='file name to save model and training record')
    parser.add_argument('--latdim', default=32, type=int, help='embedding size')
    parser.add_argument('--hyperNum', default=128, type=int, help='number of hyperedges')
    parser.add_argument('--gnn_layer', default=2, type=int, help='number of gnn layers')
    parser.add_argument('--load_model', default=None, help='model name to load')
    parser.add_argument('--topk', default=20, type=int, help='K of top K')
    parser.add_argument('--att_head', default=2, type=int, help='number of attention heads')
    parser.add_argument('--keepRate', default=0.5, type=float, help='ratio of edges to keep')
    parser.add_argument('--temp', default=0.2, type=float, help='temperature')
    parser.add_argument('--block_num', default=2, type=int, help='number of hops in gcn precessing')
    parser.add_argument('--mult', default=1, type=float, help='multiplication factor')
    parser.add_argument('--data', default='yelp', type=str, help='name of dataset')
    parser.add_argument('--tstEpoch', default=3, type=int, help='number of epoch to test while training')
    parser.add_argument('--gpu', default='0', type=str, help='indicates which gpu to use')
    parser.add_argument('--edgeSampRate', default=0.1, type=float, help='Ratio of sampled edges')
    parser.add_argument('--dropout', default=0, type=float, help='Ratio of transformer layer dropout')
    parser.add_argument('--num_head', default=4, type=int, help='Multihead number of transformer layer')
    return parser.parse_args(args=[])  # برای کولب، از args=[] استفاده می‌کنیم

args = ParseArgs()

In [5]:
# ماژول TimeLogger
logmsg = ''
timemark = dict()
saveDefault = True

def log(msg, save=None, oneline=False):
    global logmsg
    global saveDefault
    time = datetime.datetime.now()
    tem = '%s: %s' % (time, msg)
    if save is not None:
        if save:
            logmsg += tem + '\n'
    elif saveDefault:
        logmsg += tem + '\n'
    if oneline:
        print(tem, end='\r')
    else:
        print(tem)

def marktime(marker):
    global timemark
    timemark[marker] = datetime.datetime.now()


In [6]:
# ماژول Utils
def innerProduct(usrEmbeds, itmEmbeds):
    return torch.sum(usrEmbeds * itmEmbeds, dim=-1)

def pairPredict(ancEmbeds, posEmbeds, negEmbeds):
    return innerProduct(ancEmbeds, posEmbeds) - innerProduct(ancEmbeds, negEmbeds)

def calcRegLoss(model):
    ret = 0
    for W in model.parameters():
        ret += W.norm(2).square()
    return ret

def contrastLoss(embeds1, embeds2, nodes, temp):
    embeds1 = F.normalize(embeds1 + 1e-8, p=2)
    embeds2 = F.normalize(embeds2 + 1e-8, p=2)
    pckEmbeds1 = embeds1[nodes]
    pckEmbeds2 = embeds2[nodes]
    nume = torch.exp(torch.sum(pckEmbeds1 * pckEmbeds2, dim=-1) / temp)
    deno = torch.exp(pckEmbeds1 @ embeds2.T / temp).sum(-1) + 1e-8
    return -torch.log(nume / deno).mean()

In [7]:
# ماژول Transformer
def padding_mask(seq):
    length = seq.shape[1]
    mask = seq.eq(0)
    mask = mask.unsqueeze(1).expand(-1, length, -1)
    return mask

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads=8, d_ff=None, dropout=0.1, activation="relu"):
        super(TransformerEncoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.attention = nn.MultiheadAttention(d_model, num_heads, dropout=dropout)
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = getattr(F, activation)

    def forward(self, x, attn_mask=None, length_mask=None):
        N = x.shape[0]
        L = x.shape[1]

        attn_output, _ = self.attention(
            x, x, x,
            key_padding_mask=attn_mask
        )

        x = x + self.dropout(attn_output)
        y = x = self.norm1(x)
        y = self.dropout(self.activation(self.linear1(y)))
        y = self.dropout(self.linear2(y))

        return self.norm2(x + y)

class Encoder_Layer(nn.Module):
    def __init__(self, embedding_dim=128, hidden_dim=64, num_heads=8, dropout=0):
        super(Encoder_Layer, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads

        self.calculate_q = nn.Linear(self.embedding_dim, self.embedding_dim)
        torch.nn.init.xavier_normal_(self.calculate_q.weight, gain=1)
        self.calculate_k = nn.Linear(self.embedding_dim, self.embedding_dim)
        torch.nn.init.xavier_normal_(self.calculate_k.weight, gain=1)
        self.calculate_v = nn.Linear(self.embedding_dim, self.embedding_dim)
        torch.nn.init.xavier_normal_(self.calculate_v.weight, gain=1)
        self.MultiheadAttention = nn.MultiheadAttention(self.embedding_dim, num_heads, dropout=dropout)

        self.dropout = nn.Dropout(p=dropout)
        self.layer_norm = nn.LayerNorm(embedding_dim)

        self.fc = nn.Linear(embedding_dim, hidden_dim)

        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        torch.nn.init.xavier_normal_(self.fc1.weight, gain=1)

        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        torch.nn.init.xavier_normal_(self.fc2.weight, gain=1)

        self.dropout2 = nn.Dropout(p=dropout)
        self.layer_norm2 = nn.LayerNorm(hidden_dim)
        self.relu2 = nn.LeakyReLU()

    def forward(self, querys, keys, values, mask=None):
        query = self.calculate_q(querys).transpose(0, 1).contiguous()
        key = self.calculate_k(keys).transpose(0, 1).contiguous()
        value = self.calculate_v(values).transpose(0, 1).contiguous()

        output, _ = self.MultiheadAttention(query, key, value, key_padding_mask=mask)
        output = output.transpose(0, 1).contiguous()

        output = querys + self.dropout(output)
        output = self.layer_norm(output)

        output = self.fc(output)
        tmp = output

        output = self.fc2(self.dropout2(self.relu2(self.fc1(output))))
        output = tmp + self.dropout2(output)
        output = self.layer_norm2(output)

        return output

In [8]:
# مدل TransGNN
class TransGNN(nn.Module):
    def __init__(self):
        super(TransGNN, self).__init__()
        self.user_embeding = nn.Parameter(nn.init.xavier_uniform_(torch.empty(args.user, args.latdim)))
        self.item_embeding = nn.Parameter(nn.init.xavier_uniform_(torch.empty(args.item, args.latdim)))
        self.user_transformer_encoder = TransformerEncoderLayer(d_model=args.latdim, num_heads=args.num_head, dropout=args.dropout)
        self.item_transformer_encoder = TransformerEncoderLayer(d_model=args.latdim, num_heads=args.num_head, dropout=args.dropout)

    def user_transformer_layer(self, embeds, mask=None):
        assert len(embeds.shape) <= 3, f"Shape Error, embed shape is {embeds.shape}, out of size!"
        if len(embeds.shape) == 2:
            embeds = embeds.unsqueeze(dim=0)
            embeds = self.user_transformer_encoder(embeds, mask)
            embeds = embeds.squeeze()
        else:
            embeds = self.user_transformer_encoder(embeds, mask)
        return embeds

    def item_transformer_layer(self, embeds, mask=None):
        assert len(embeds.shape) <= 3, f"Shape Error, embed shape is {embeds.shape}, out of size!"
        if len(embeds.shape) == 2:
            embeds = embeds.unsqueeze(dim=0)
            embeds = self.item_transformer_encoder(embeds, mask)
            embeds = embeds.squeeze()
        else:
            embeds = self.item_transformer_encoder(embeds, mask)
        return embeds

    def gnn_message_passing(self, adj, embeds):
        return torch.spmm(adj, embeds)

    def forward(self, adj):
        embeds = [torch.concat([self.user_embeding, self.item_embeding], dim=0)]
        for i in range(args.block_num):
            tmp_embeds = self.gnn_message_passing(adj, embeds[-1])
            tmp_user_embeds = tmp_embeds[:args.user]
            tmp_item_embeds = tmp_embeds[args.user:]
            tmp_user_embeds = self.user_transformer_layer(tmp_user_embeds)
            tmp_item_embeds = self.item_transformer_layer(tmp_item_embeds)

            tmp_user_embeds += tmp_embeds[:args.user]
            tmp_item_embeds += tmp_embeds[args.user:]

            tmp_embeds = torch.concat([tmp_user_embeds, tmp_item_embeds], dim=0)
            embeds.append(tmp_embeds)

        embeds = sum(embeds)
        user_embeds = embeds[:args.user]
        item_embeds = embeds[args.user:]
        return embeds, user_embeds, item_embeds

    def pickEdges(self, adj):
        idx = adj._indices()
        rows, cols = idx[0, :], idx[1, :]
        mask = torch.logical_and(rows <= args.user, cols > args.user)
        rows, cols = rows[mask], cols[mask]
        edgeSampNum = int(args.edgeSampRate * rows.shape[0])
        if edgeSampNum % 2 == 1:
            edgeSampNum += 1
        edgeids = torch.randint(rows.shape[0], [edgeSampNum])
        pckUsrs, pckItms = rows[edgeids], cols[edgeids] - args.user
        return pckUsrs, pckItms

    def pickRandomEdges(self, adj):
        edgeNum = adj._indices().shape[1]
        edgeSampNum = int(args.edgeSampRate * edgeNum)
        if edgeSampNum % 2 == 1:
            edgeSampNum += 1
        rows = torch.randint(args.user, [edgeSampNum])
        cols = torch.randint(args.item, [edgeSampNum])
        return rows, cols

    def bprLoss(self, user_embeding, item_embeding, ancs, poss, negs):
        ancEmbeds = user_embeding[ancs]
        posEmbeds = item_embeding[poss]
        negEmbeds = item_embeding[negs]
        scoreDiff = pairPredict(ancEmbeds, posEmbeds, negEmbeds)
        bprLoss = - ((scoreDiff).sigmoid() + 1e-6).log().mean()
        return bprLoss

    def calcLosses(self, ancs, poss, negs, adj):
        embeds, user_embeds, item_embeds = self.forward(adj)
        user_embeding, item_embeding = embeds[:args.user], embeds[args.user:]
        bprLoss = self.bprLoss(user_embeding, item_embeding, ancs, poss, negs) + self.bprLoss(user_embeds, item_embeds, ancs, poss, negs)
        return bprLoss

    def predict(self, adj):
        embeds, user_embeds, item_embeds = self.forward(adj)
        return user_embeds, item_embeds

In [9]:
# کلاس DataHandler
class DataHandler:
    def __init__(self):
        if args.data == 'yelp':
            predir = 'Data/yelp/'
        elif args.data == 'ml10m':
            predir = 'Data/ml10m/'
        elif args.data == 'tmall':
            predir = 'Data/tmall/'
        elif args.data == 'gowalla':
            predir = 'Data/gowalla/'
        elif args.data == 'amazon-book':
            predir = 'Data/amazon-book/'
        self.predir = predir
        self.trnfile = predir + 'trnMat.pkl'
        self.tstfile = predir + 'tstMat.pkl'

    def loadOneFile(self, filename):
        with open(filename, 'rb') as fs:
            ret = (pickle.load(fs) != 0).astype(np.float32)
        if type(ret) != coo_matrix:
            ret = sp.coo_matrix(ret)
        return ret

    def normalizeAdj(self, mat):
        degree = np.array(mat.sum(axis=-1))
        dInvSqrt = np.reshape(np.power(degree, -0.5), [-1])
        dInvSqrt[np.isinf(dInvSqrt)] = 0.0
        dInvSqrtMat = sp.diags(dInvSqrt)
        return mat.dot(dInvSqrtMat).transpose().dot(dInvSqrtMat).tocoo()

    def makeTorchAdj(self, mat):
        a = sp.csr_matrix((args.user, args.user))
        b = sp.csr_matrix((args.item, args.item))
        mat = sp.vstack([sp.hstack([a, mat]), sp.hstack([mat.transpose(), b])])
        mat = (mat != 0) * 1.0
        mat = self.normalizeAdj(mat)
        idxs = torch.from_numpy(np.vstack([mat.row, mat.col]).astype(np.int64))
        vals = torch.from_numpy(mat.data.astype(np.float32))
        shape = torch.Size(mat.shape)
        return torch.sparse.FloatTensor(idxs, vals, shape).cuda()

    def makeSample(self):
        user_sample_idx = torch.tensor([[args.user + i for i in range(args.item)] * args.user])
        item_sample_idx = torch.tensor([[i for i in range(args.user)] * args.item])
        return user_sample_idx, item_sample_idx

    def makeMask(self):
        u_u_mask = torch.zeros(size=(args.user, args.user), dtype=bool)
        u_i_mask = torch.ones(size=(args.user, args.item), dtype=bool)
        i_i_mask = torch.zeros(size=(args.item, args.item), dtype=bool)
        i_u_mask = torch.ones(size=(args.item, args.user), dtype=bool)
        u_mask = torch.concat([u_u_mask, u_i_mask], dim=-1)
        i_mask = torch.concat([i_u_mask, i_i_mask], dim=-1)
        mask = torch.concat([u_mask, i_mask], dim=0)
        return mask

    def LoadData(self):
        trnMat = self.loadOneFile(self.trnfile)
        tstMat = self.loadOneFile(self.tstfile)
        args.user, args.item = trnMat.shape
        self.torchBiAdj = self.makeTorchAdj(trnMat)
        self.mask = self.makeMask()
        trnData = TrnData(trnMat)
        self.trnLoader = data.DataLoader(trnData, batch_size=args.batch, shuffle=True, num_workers=0)
        tstData = TstData(tstMat, trnMat)
        self.tstLoader = data.DataLoader(tstData, batch_size=args.tstBat, shuffle=False, num_workers=0)

In [10]:
# کلاس‌های Dataset
class TrnData(data.Dataset):
    def __init__(self, coomat):
        self.rows = coomat.row
        self.cols = coomat.col
        self.dokmat = coomat.todok()
        self.negs = np.zeros(len(self.rows)).astype(np.int32)

    def negSampling(self):
        for i in range(len(self.rows)):
            u = self.rows[i]
            while True:
                iNeg = np.random.randint(args.item)
                if (u, iNeg) not in self.dokmat:
                    break
            self.negs[i] = iNeg

    def __len__(self):
        return len(self.rows)

    def __getitem__(self, idx):
        return self.rows[idx], self.cols[idx], self.negs[idx]

class TstData(data.Dataset):
    def __init__(self, coomat, trnMat):
        self.csrmat = (trnMat.tocsr() != 0) * 1.0
        tstLocs = [None] * coomat.shape[0]
        tstUsrs = set()
        for i in range(len(coomat.data)):
            row = coomat.row[i]
            col = coomat.col[i]
            if tstLocs[row] is None:
                tstLocs[row] = list()
            tstLocs[row].append(col)
            tstUsrs.add(row)
        tstUsrs = np.array(list(tstUsrs))
        self.tstUsrs = tstUsrs
        self.tstLocs = tstLocs

    def __len__(self):
        return len(self.tstUsrs)

    def __getitem__(self, idx):
        return self.tstUsrs[idx], np.reshape(self.csrmat[self.tstUsrs[idx]].toarray(), [-1])

In [11]:
# کلاس Coach
class Coach:
    def __init__(self, handler):
        self.handler = handler
        print('USER', args.user, 'ITEM', args.item)
        print('NUM OF INTERACTIONS', self.handler.trnLoader.dataset.__len__())
        self.metrics = dict()
        mets = ['Loss', 'preLoss', 'Recall', 'NDCG']
        for met in mets:
            self.metrics['Train' + met] = list()
            self.metrics['Test' + met] = list()

    def makePrint(self, name, ep, reses, save):
        ret = 'Epoch %d/%d, %s: ' % (ep, args.epoch, name)
        for metric in reses:
            val = reses[metric]
            ret += '%s = %.4f, ' % (metric, val)
            tem = name + metric
            if save and tem in self.metrics:
                self.metrics[tem].append(val)
        ret = ret[:-2] + '  '
        return ret

    def run(self):
        self.prepareModel()
        log('Model Prepared')
        if args.load_model is not None:
            self.loadModel()
            stloc = len(self.metrics['TrainLoss']) * args.tstEpoch - (args.tstEpoch - 1)
        else:
            stloc = 0
            log('Model Initialized')
        for ep in range(stloc, args.epoch):
            tstFlag = (ep % args.tstEpoch == 0)
            reses = self.trainEpoch()
            log(self.makePrint('Train', ep, reses, tstFlag))
            if tstFlag:
                reses = self.testEpoch()
                log(self.makePrint('Test', ep, reses, tstFlag))
                self.saveHistory()
            print()
        reses = self.testEpoch()
        log(self.makePrint('Test', args.epoch, reses, True))
        self.saveHistory()

    def prepareModel(self):
        self.model = TransGNN().cuda()
        self.opt = torch.optim.Adam(self.model.parameters(), lr=args.lr, weight_decay=0)

    def trainEpoch(self):
        trnLoader = self.handler.trnLoader
        trnLoader.dataset.negSampling()
        epLoss, epPreLoss = 0, 0
        steps = trnLoader.dataset.__len__() // args.batch
        for i, tem in enumerate(trnLoader):
            ancs, poss, negs = tem
            ancs = ancs.long().cuda()
            poss = poss.long().cuda()
            negs = negs.long().cuda()
            bprLoss = self.model.calcLosses(ancs, poss, negs, self.handler.torchBiAdj)
            loss = bprLoss
            epLoss += loss.item()
            epPreLoss += bprLoss.item()
            self.opt.zero_grad()
            loss.backward()
            self.opt.step()
            log('Step %d/%d: loss = %.3f         ' % (i, steps, loss), save=False, oneline=True)
        ret = dict()
        ret['Loss'] = epLoss / steps
        ret['preLoss'] = epPreLoss / steps
        return ret

    def testEpoch(self):
        tstLoader = self.handler.tstLoader
        epLoss, epRecall, epNdcg = [0] * 3
        i = 0
        num = tstLoader.dataset.__len__()
        steps = num // args.tstBat
        for usr, trnMask in tstLoader:
            i += 1
            usr = usr.long().cuda()
            trnMask = trnMask.cuda()
            usrEmbeds, itmEmbeds = self.model.predict(self.handler.torchBiAdj)
            allPreds = torch.mm(usrEmbeds[usr], torch.transpose(itmEmbeds, 1, 0)) * (1 - trnMask) - trnMask * 1e8
            _, topLocs = torch.topk(allPreds, args.topk)
            recall, ndcg = self.calcRes(topLocs.cpu().numpy(), self.handler.tstLoader.dataset.tstLocs, usr)
            epRecall += recall
            epNdcg += ndcg
            log('Steps %d/%d: recall = %.2f, ndcg = %.2f          ' % (i, steps, recall, ndcg), save=False, oneline=True)
        ret = dict()
        ret['Recall'] = epRecall / num
        ret['NDCG'] = epNdcg / num
        return ret

    def calcRes(self, topLocs, tstLocs, batIds):
        assert topLocs.shape[0] == len(batIds)
        allRecall = allNdcg = 0
        for i in range(len(batIds)):
            temTopLocs = list(topLocs[i])
            temTstLocs = tstLocs[batIds[i]]
            tstNum = len(temTstLocs)
            maxDcg = np.sum([np.reciprocal(np.log2(loc + 2)) for loc in range(min(tstNum, args.topk))])
            recall = dcg = 0
            for val in temTstLocs:
                if val in temTopLocs:
                    recall += 1
                    dcg += np.reciprocal(np.log2(temTopLocs.index(val) + 2))
            recall = recall / tstNum
            ndcg = dcg / maxDcg
            allRecall += recall
            allNdcg += ndcg
        return allRecall, allNdcg

    def saveHistory(self):
        if args.epoch == 0:
            return
        os.makedirs('History', exist_ok=True)
        with open('History/' + args.save_path + '.his', 'wb') as fs:
            pickle.dump(self.metrics, fs)
        os.makedirs('Models', exist_ok=True)
        content = {'model': self.model}
        torch.save(content, 'Models/' + args.save_path + '.mod')
        log('Model Saved: %s' % args.save_path)

    def loadModel(self):
        ckp = torch.load('Models/' + args.load_model + '.mod')
        self.model = ckp['model']
        self.opt = torch.optim.Adam(self.model.parameters(), lr=args.lr, weight_decay=0)
        with open('History/' + args.load_model + '.his', 'rb') as fs:
            self.metrics = pickle.load(fs)
        log('Model Loaded')

In [ ]:
# اجرای اصلی
if __name__ == '__main__':
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    import setproctitle
    setproctitle.setproctitle('proc_title')

    log('Start')
    handler = DataHandler()
    handler.LoadData()
    log('Load Data')
    coach = Coach(handler)
    coach.run()

2025-05-19 07:57:30.888912: Start


<ipython-input-9-5a515d09767f>:20: DeprecationWarning: Please import `coo_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.coo` namespace is deprecated and will be removed in SciPy 2.0.0.
  ret = (pickle.load(fs) != 0).astype(np.float32)
<ipython-input-9-5a515d09767f>:20: DeprecationWarning: numpy.core._multiarray_umath is deprecated and has been renamed to numpy._core._multiarray_umath. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core._multiarray_umath._reconstruct.
  ret = (pickle.load(fs) != 0).astype(np.float32)
<ipython-input-9-5a515d09767f>:41: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, devic

2025-05-19 07:57:39.913322: Load Data
USER 29601 ITEM 24734
NUM OF INTERACTIONS 1069128
2025-05-19 07:57:39.930747: Model Prepared
2025-05-19 07:57:39.930778: Model Initialized
2025-05-19 07:58:09.199556: Epoch 0/40, Train: Loss = 0.3216, preLoss = 0.3216  
2025-05-19 07:58:34.295860: Epoch 0/40, Test: Recall = 0.0646, NDCG = 0.0530  
2025-05-19 07:58:34.314567: Model Saved: tem

2025-05-19 07:59:04.071486: Epoch 1/40, Train: Loss = 0.1681, preLoss = 0.1681  

2025-05-19 07:59:32.806635: Epoch 2/40, Train: Loss = 0.1442, preLoss = 0.1442  

2025-05-19 08:00:02.504283: Epoch 3/40, Train: Loss = 0.1317, preLoss = 0.1317  
2025-05-19 08:00:27.655218: Epoch 3/40, Test: Recall = 0.0762, NDCG = 0.0651  
2025-05-19 08:00:27.672751: Model Saved: tem

2025-05-19 08:00:56.509420: Epoch 4/40, Train: Loss = 0.1191, preLoss = 0.1191  

2025-05-19 08:01:25.474755: Epoch 5/40, Train: Loss = 0.1125, preLoss = 0.1125  

2025-05-19 08:01:55.329123: Epoch 6/40, Train: Loss = 0.1095, preLoss = 0.1095  
20